In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [5]:
from toolz import partial
import pickle
import optuna

In [6]:
from xgboost import XGBClassifier

In [7]:
def rename_columns(path:str = '../data/raw/heart.csv'):# set columns as constants
    NUMERICAL_COLUMNS = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
    CATEGORICAL_COLUMNS = ["FastingBS", 'Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
    TARGET_COLUMN = 'HeartDisease'
    data = pd.read_csv(path)
    data = data[CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS + [TARGET_COLUMN]]
    data.columns = [
        "fasting_bs", "sex", "chest_pain_type", "resting_ecg", "exercise_angina", "st_slope", 
        'age', 'resting_bp', 'cholesterol', 'max_heart_rate', 'old_peak',
        'heart_disease'
    ]
    data.to_csv('../data/heart.csv', index=False)
    return data

In [8]:
def reconstruct(transformed_data, preprocessor):
    return pd.DataFrame(transformed_data, columns=preprocessor.transformers_[0][1].get_feature_names_out().tolist() + NUMERICAL_COLUMNS)

def preprocess(X, preprocessor, fit=False):
    if (fit):
        preprocessor.fit(X)
    return reconstruct(preprocessor.transform(X), preprocessor), preprocessor

In [24]:
CATEGORICAL_COLUMNS = ["fasting_bs", "sex", "chest_pain_type", "resting_ecg", "exercise_angina", "st_slope" ]
NUMERICAL_COLUMNS = ['cholesterol', 'age', 'resting_bp', 'max_heart_rate', 'old_peak']
TARGET_COLUMN = 'heart_disease'
SEED = 42

def main(PREPROCESSOR_FILE = "../data/models/preprocessor.bin", LR_FILE = "../data/models/lr.bin", RF_FILE = "../data/models/rf.bin", XGB_FILE = "../data/models/xgb.bin"):
    
    # get data
    data = rename_columns()
    
    # split test-train-val sets
    X = data[CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS]
    y = data[TARGET_COLUMN]
    X_full_train, X_test, y_full_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=SEED)
    X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size = 0.25)
    
    #define preprocessing step
    onehot_preprocessor = OneHotEncoder(handle_unknown="ignore")
    cholesterol_imputer = Pipeline(steps=[("imputer", SimpleImputer(missing_values=0, strategy="mean"))])
    preprocessor = ColumnTransformer(
        transformers=[
            ("categorical", onehot_preprocessor, CATEGORICAL_COLUMNS),
            ("cholesterol_processor", cholesterol_imputer, ["cholesterol"]),
        ],
        remainder='passthrough'
    )
    
    #preprocess for training
    X_train, preprocessor = preprocess(X_train, preprocessor, fit=True)
    X_val, _preprocessor = preprocess(X_val, preprocessor)

    lr_best_params = evaluate_lr(X_train, y_train, X_val, y_val)
    rf_best_params = evaluate_rf(X_train, y_train, X_val, y_val)
    xgb_best_params = evaluate_xgb(X_train, y_train, X_val, y_val)
    
    #refit preprocessor
    X_full_train, preprocessor = preprocess(X_full_train, preprocessor, fit=True)
    X_test, _preprocessor = preprocess(X_test, preprocessor)

    pickle.dump(preprocessor, open(PREPROCESSOR_FILE, 'wb'))
    
    ## TODO: put these lines on a foreach
    # LR retrain
    model = LogisticRegression(random_state=SEED, **lr_best_params)
    model.fit(X_full_train, y_full_train)
    print("LR params", lr_best_params)
    print("LR Accuracy (train, test):", accuracy_score(y_full_train, model.predict(X_full_train)), accuracy_score(y_test, model.predict(X_test)))
    pickle.dump(model, open(LR_FILE, 'wb'))
    
    # RF retraing
    model = RandomForestClassifier(random_state=SEED, **rf_best_params)
    model.fit(X_full_train, y_full_train)
    print("RF params", rf_best_params)
    print("RF Accuracy (train, test):", accuracy_score(y_full_train, model.predict(X_full_train)), accuracy_score(y_test, model.predict(X_test)))
    pickle.dump(model, open(RF_FILE, 'wb'))

    # XGB retrain
    model = XGBClassifier(**xgb_best_params, **{"eval_metric": accuracy_score, "verbosity": 0})  
    model.fit(X_full_train, y_full_train, eval_set=[(X_full_train, y_full_train), (X_test, y_test)])
    print("XGB params", xgb_best_params)
    print("XGBClassifier Accuracy (train, test):", accuracy_score(y_full_train, model.predict(X_full_train)), accuracy_score(y_test, model.predict(X_test)))

    pickle.dump(model, open(XGB_FILE, 'wb'))

In [25]:
def evaluate_lr(X_train, y_train, X_val, y_val):
    def objective(trial, X_train, y_train, X_val, y_val):
        C = trial.suggest_loguniform("C", 1e-7, 10.0)
        solver = trial.suggest_categorical("solver", ("lbfgs", "saga"))

        clf = LogisticRegression(C=C, solver=solver)
        
        clf.fit(X_train, y_train)

        return accuracy_score(y_val, clf.predict(X_val))

    sampler=optuna.samplers.TPESampler(seed=SEED)
    study = optuna.create_study(direction="maximize")
    study.optimize(partial(objective, X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val), n_trials=10)
    
    return study.best_params

In [26]:
def evaluate_rf(X_train, y_train, X_val, y_val):
    
    def objective_rf(trial, X_train, y_train, X_val, y_val):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'max_depth': trial.suggest_int('max_depth', 4, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 150),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 60),
        }

        model = RandomForestClassifier(random_state=SEED, **params)

        model.fit(X_train, y_train)

        return accuracy_score(y_val, model.predict(X_val))


    sampler=optuna.samplers.TPESampler(seed=SEED)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(partial(objective_rf, X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val), n_trials=20)
    
    rf_best_params = study.best_params
    return rf_best_params

In [27]:
def evaluate_xgb(X_train, y_train, X_val, y_val):
    def objective_xgb(trial, X_train, y_train, X_val, y_val):
 
        param = {
            "eval_metric": accuracy_score,
            "early_stopping_rounds": 20,
            "verbosity": 0,
            # use exact for small dataset.
            "tree_method": "exact",
            # L2 regularization weight.
            "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
            # L1 regularization weight.
            "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
            # sampling ratio for training data.
            "subsample": trial.suggest_float("subsample", 0.2, 1.0),
            # sampling according to each tree.
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 9, step=2),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0, log=True)
        }

        model = XGBClassifier(**param)  

        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], )

        return accuracy_score(y_val, model.predict(X_val))
    
    sampler=optuna.samplers.TPESampler(seed=SEED)
    study_xgb = optuna.create_study(direction='maximize', sampler=sampler)
    study_xgb.optimize(partial(objective_xgb, X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val), n_trials=20)
    xgb_best_params = study_xgb.best_params
    return xgb_best_params

In [28]:
main()

[I 2021-11-01 02:15:06,655] A new study created in memory with name: no-name-38be5566-960b-46c8-a9c8-5dd3bda77a10
/home/jloscalzo/.pyenv/versions/bolsa_gatos_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2021-11-01 02:15:06,691] Trial 0 finished with value: 0.7010869565217391 and parameters: {'C': 0.6744040295539879, 'solver': 'saga'}. Best is trial 0 with value: 0.7010869565217391.
/home/jloscalzo/.pyenv/versions/bolsa_gatos_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#

[0]	validation_0-logloss:0.69156	validation_1-logloss:0.69158
[1]	validation_0-logloss:0.68998	validation_1-logloss:0.69002
[2]	validation_0-logloss:0.68837	validation_1-logloss:0.68842
[3]	validation_0-logloss:0.68681	validation_1-logloss:0.68708
[4]	validation_0-logloss:0.68522	validation_1-logloss:0.68552
[5]	validation_0-logloss:0.68365	validation_1-logloss:0.68421
[6]	validation_0-logloss:0.68211	validation_1-logloss:0.68279
[7]	validation_0-logloss:0.68062	validation_1-logloss:0.68131


/home/jloscalzo/.pyenv/versions/bolsa_gatos_env/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[8]	validation_0-logloss:0.67909	validation_1-logloss:0.67976
[9]	validation_0-logloss:0.67771	validation_1-logloss:0.67836
[10]	validation_0-logloss:0.67624	validation_1-logloss:0.67689
[11]	validation_0-logloss:0.67472	validation_1-logloss:0.67559
[12]	validation_0-logloss:0.67323	validation_1-logloss:0.67419
[13]	validation_0-logloss:0.67180	validation_1-logloss:0.67275
[14]	validation_0-logloss:0.67034	validation_1-logloss:0.67139
[15]	validation_0-logloss:0.66886	validation_1-logloss:0.66993
[16]	validation_0-logloss:0.66748	validation_1-logloss:0.66848
[17]	validation_0-logloss:0.66605	validation_1-logloss:0.66705
[18]	validation_0-logloss:0.66462	validation_1-logloss:0.66570
[19]	validation_0-logloss:0.66314	validation_1-logloss:0.66437
[20]	validation_0-logloss:0.66179	validation_1-logloss:0.66324
[21]	validation_0-logloss:0.66036	validation_1-logloss:0.66179
[22]	validation_0-logloss:0.65903	validation_1-logloss:0.66064
[23]	validation_0-logloss:0.65763	validation_1-logloss:0.

[I 2021-11-01 02:15:32,081] Trial 0 finished with value: 0.8532608695652174 and parameters: {'lambda': 9.915644566638385e-06, 'alpha': 0.40338008326003827, 'subsample': 0.7855951534491241, 'colsample_bytree': 0.6789267873576292, 'max_depth': 3, 'learning_rate': 0.0029375384576328283}. Best is trial 0 with value: 0.8532608695652174.


...

[10]	validation_0-logloss:0.67412	validation_1-logloss:0.67433
[11]	validation_0-logloss:0.67231	validation_1-logloss:0.67266
[12]	validation_0-logloss:0.67044	validation_1-logloss:0.67085
[13]	validation_0-logloss:0.66876	validation_1-logloss:0.66944
[14]	validation_0-logloss:0.66723	validation_1-logloss:0.66797
[15]	validation_0-logloss:0.66529	validation_1-logloss:0.66602
[16]	validation_0-logloss:0.66346	validation_1-logloss:0.66397
[17]	validation_0-logloss:0.66179	validation_1-logloss:0.66233
[18]	validation_0-logloss:0.65984	validation_1-logloss:0.66057
[19]	validation_0-logloss:0.65797	validation_1-logloss:0.65855
[20]	validation_0-logloss:0.65646	validation_1-logloss:0.65727
[21]	validation_0-logloss:0.65469	validation_1-logloss:0.65560
[22]	validation_0-logloss:0.65285	validation_1-logloss:0.65419
[23]	validation_0-logloss:0.65116	validation_1-logloss:0.65249
[24]	validation_0-logloss:0.64992	validation_1-logloss:0.65114
[25]	validation_0-logloss:0.64823	validation_1-logloss:

[I 2021-11-01 02:16:04,333] Trial 19 finished with value: 0.8532608695652174 and parameters: {'lambda': 0.004441303161142579, 'alpha': 0.00017892683907651672, 'subsample': 0.20222625730468002, 'colsample_bytree': 0.5483749202654314, 'max_depth': 7, 'learning_rate': 0.0035407805304457684}. Best is trial 10 with value: 0.9130434782608695.
/home/jloscalzo/.pyenv/versions/bolsa_gatos_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR params {'C': 0.8512868983699925, 'solver': 'lbfgs'}
LR Accuracy (train, test): 0.8651226158038147 0.8695652173913043
RF params {'n_estimators': 758, 'max_depth': 25, 'min_samples_split': 53, 'min_samples_leaf': 15}
RF Accuracy (train, test): 0.8651226158038147 0.842391304347826
[0]	validation_0-logloss:0.68909	validation_1-logloss:0.68948
[1]	validation_0-logloss:0.68725	validation_1-logloss:0.68726
[2]	validation_0-logloss:0.68339	validation_1-logloss:0.68417
[3]	validation_0-logloss:0.67965	validation_1-logloss:0.68054
[4]	validation_0-logloss:0.67667	validation_1-logloss:0.67855
[5]	validation_0-logloss:0.67545	validation_1-logloss:0.67785
[6]	validation_0-logloss:0.67192	validation_1-logloss:0.67510
[7]	validation_0-logloss:0.66917	validation_1-logloss:0.67257


/home/jloscalzo/.pyenv/versions/bolsa_gatos_env/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[8]	validation_0-logloss:0.66765	validation_1-logloss:0.67115
[9]	validation_0-logloss:0.66598	validation_1-logloss:0.66959
[10]	validation_0-logloss:0.66369	validation_1-logloss:0.66716
[11]	validation_0-logloss:0.66076	validation_1-logloss:0.66435
[12]	validation_0-logloss:0.65851	validation_1-logloss:0.66320
[13]	validation_0-logloss:0.65678	validation_1-logloss:0.66155
[14]	validation_0-logloss:0.65398	validation_1-logloss:0.65875
[15]	validation_0-logloss:0.65146	validation_1-logloss:0.65692
[16]	validation_0-logloss:0.64992	validation_1-logloss:0.65633
[17]	validation_0-logloss:0.64738	validation_1-logloss:0.65413
[18]	validation_0-logloss:0.64465	validation_1-logloss:0.65109
[19]	validation_0-logloss:0.64136	validation_1-logloss:0.64832
[20]	validation_0-logloss:0.63877	validation_1-logloss:0.64612
[21]	validation_0-logloss:0.63662	validation_1-logloss:0.64488
[22]	validation_0-logloss:0.63326	validation_1-logloss:0.64217
[23]	validation_0-logloss:0.63084	validation_1-logloss:0.